In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import requests
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf

import time

import os
from os import listdir
from os.path import isfile, join

import statsmodels.api as sm
import seaborn as sns

from statsmodels.tsa.ar_model import AutoReg, ar_select_order

In [2]:
pip install mplfinance

  Obtaining dependency information for mplfinance from https://files.pythonhosted.org/packages/d7/d9/31c436ea7673c21a5bf3fc747bc7f63377582dfe845c3004d3e46f9deee0/mplfinance-0.12.10b0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/75.0 kB ? eta -:--:--
   ---------------------------------------- 75.0/75.0 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


In [6]:
#Some Defatult Values
PATH = "C:/Users/dietd/Untitled Folder/Python for Finance/Financial Data/Stock Data/"
start_date = "2020-01-02"
today = dt.datetime.today().strftime("%Y-%m-%d")
print("Start date: ",start_date)
print("Today's date: ", today)

Start date:  2020-01-02
Today's date:  2024-09-24


In [7]:
NYSE_df = pd.read_csv("NYSE_listed.csv")
NYSE_df.head(50)

,ACT Symbol,Company Name
0,A,"Agilent Technologies, Inc. Common Stock"
1,AA,Alcoa Inc. Common Stock
2,AA$B,Alcoa Inc. Depository Shares Representing 1/10...
3,AAC,"AAC Holdings, Inc. Common Stock"
4,AAN,"Aaron's, Inc. Common Stock"
5,AAP,Advance Auto Parts Inc Advance Auto Parts Inc W/I
6,AAT,"American Assets Trust, Inc. Common Stock"
7,AAV,Advantage Oil & Gas Ltd Ordinary Shares
8,AB,Allianceberstein Holding L.P. Units
9,ABB,ABB Ltd Common Stock


In [8]:
NYSE_tickers = NYSE_df["ACT Symbol"]
NYSE_tickers
NYSE_tickers_short = NYSE_tickers[100::500]
stocks_not_downloaded = []
own_tickers = ["META", "AAPL", "NFLX", "TSLA", "AMZN", "GOOG", "AMD", "^GSPC"]
print(len(NYSE_tickers_short))

7


In [9]:
beursrally_tickers = pd.read_csv('BeursRally_Stock.csv')
euronext_brussels = pd.read_csv('EuronextBrussel.csv')
#display(beursrally_tickers['Company Name'][0])
#display(euronext_brussels['Name'])
beursrally_brussel = []
for name in beursrally_tickers['Company Name']:
    if name in list(euronext_brussels['Name']):
        index = list(euronext_brussels['Name']).index(name)
        beursrally_brussel.append(euronext_brussels.loc[index, 'Symbol'])

#display(beursrally_brussel)

In [10]:
euronext_amsterdam = pd.read_csv('Amsterdam.csv')
beursrally_amsterdam = []
for name in beursrally_tickers['Company Name']:
    if name in list(euronext_amsterdam['Name']):
        index = list(euronext_amsterdam['Name']).index(name)
        beursrally_amsterdam.append(euronext_amsterdam.loc[index, 'Symbol'])


In [11]:
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
ny_tickers = []
ny_names = []
for l in letters:
    l_list = list(pd.read_html('https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_('+l+')')[0].Symbol)
    name_list = list(pd.read_html('https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_('+l+')')[0]['Stock name'])
    for i in range(len(l_list)):
        ny_tickers.append(l_list[i])
        ny_names.append(name_list[i])
ny_names

['A. O. Smith Corporation',
 'A10 Networks, Inc.',
 'AAC Holdings Inc.',
 'AAR Corporation',
 "Aaron's Inc.",
 'ABB LTD.',
 'Abbott Laboratories',
 'AbbVie Inc.',
 'Abercrombie & Fitch',
 'Aberdeen Greater China Fund, Inc.',
 'Aberdeen Japan Equity Fund, Inc.',
 'Aberdeen Singapore Fund, Inc.',
 'ABM Industries Incorporated',
 'Acadia Realty Trust',
 'Accenture plc',
 'ACCO Brands Corporation',
 'Acorn International, Inc.',
 'Actuant Corporation',
 'Acushnet Holdings Corp.',
 'Acuity Brands, Inc.',
 'Adams Express Company',
 'Adecoagro S.A.',
 'Adeptus Health Inc.',
 'Adient Plc',
 'Advance Auto Parts, Inc.',
 'Advanced Disposal Services Inc.',
 'Advanced Drainage Systems Inc.',
 'Advanced Semiconductor Engineering, Inc.',
 'AdvancePierre Food Holdings Inc.',
 'AdvanSix Inc.',
 'Advantage Oil & Gas Ltd.',
 'Advent Claymore Convertible Securities and Income Fund',
 'Advent Claymore Convertible Securities and Income Fund II',
 'Advent Claymore Enhanced Growth & Income Fund',
 'AECOM Tech

In [12]:
beursrally_ny = []
for name in beursrally_tickers['Company Name']:
    if name in name_list:
        index = name_list.index(name)
        beursrally_ny.append(ny_tickers[index])

beursrally_ny.append('MMM')

In [13]:
euronext_paris = pd.read_csv('Paris.csv')
beursrally_paris = []
for name in beursrally_tickers['Company Name']:
    if name in list(euronext_paris['Name']):
        index = list(euronext_paris['Name']).index(name)
        beursrally_paris.append(euronext_paris.loc[index, 'Symbol'])


In [14]:
nasdaq = pd.read_csv('NASDAQ.csv')
beursrally_nasdaq = []
for name in beursrally_tickers['Company Name']:
    if name in list(nasdaq['Name']):
        index = list(nasdaq['Name']).index(name)
        beursrally_nasdaq.append(nasdaq.loc[index, 'Symbol'])

In [15]:
BeursRallyComplete = beursrally_brussel+beursrally_amsterdam+beursrally_nasdaq+beursrally_ny+beursrally_paris+own_tickers
templist = list(dict.fromkeys(BeursRallyComplete))
BeursRallyComplete = templist
BeursRallyComplete.remove('ATO')
BeursRallyComplete.remove('DG')
BeursRallyComplete.remove('FR')
BeursRallyComplete.remove('GNE')

In [16]:
test_df = yf.download(BeursRallyComplete, start_date)['Adj Close']
test_df = test_df.dropna(axis=1)
test_df

[*********************100%***********************]  147 of 147 completed

96 Failed downloads:
['TWEKA', 'VALUE', 'AGN', 'NEDAP', 'STMPA', 'TEXF', 'CS', 'WEHB', 'BCART', 'DECB', 'BOLS', 'SHUR', 'OBEL', 'VRAP', 'AGFB', 'SEQUA', 'AKE', 'TINC', 'KBCA', 'SBMO', 'EKOP', 'ADYEN', 'MITRA', 'COLR', 'AKZA', 'IMCD', 'BTLS', 'FFARM', 'GIMB', 'IBAB', 'BEKB', 'ARCAD', 'ONWD', 'MELE', 'XIOR', 'CAP', 'SIFG', 'ENGI', 'NEX', 'IMMO', 'CO', 'HEIA', 'LOTB', 'ASCE', 'FAGR', 'TESB', 'GBLB', 'PHARM', 'ASRNL', 'BPOST', 'ATEB', 'XFAB', 'BBED', 'BREB', 'GREEN', 'TKTT', 'OXUR', 'COFB', 'SOLB', 'CFEB', 'CYAD', 'SLIGR', 'VASTN', 'NK', 'TOM2', 'KENDR']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['RI', 'PNL', 'VGP', 'JEN', 'UNA', 'KER', 'FUR', 'WDP', 'OCI', 'SIP', 'HO', 'PROX', 'EXM', 'RNO', 'AED', 'ROU', 'HYL', 'LR', 'WHA', 'VK', 'MONT', 'ABI', 'ORP', 'VAN', 'BESI', 'EN', 'VPK', 'WKL', 'RET', 'QRF']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-01-02 ->

Ticker,AAPL,AC,AGS,AIR,AMD,AMG,AMZN,APAM,ASM,ASML,...,ORA,SAN,SAVE,SLB,SMAR,SU,TSLA,UCB,UMI,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02 00:00:00+00:00,72.876106,38.339832,12.120,45.750000,49.099998,83.546501,94.900497,21.727211,0.62,290.156311,...,72.670090,3.845528,36.785278,36.206539,45.840000,26.762108,28.684000,26.747797,22.759552,3257.850098
2020-01-03 00:00:00+00:00,72.167587,38.349533,11.570,45.709999,48.599998,82.512009,93.748497,21.707165,0.64,285.480530,...,72.543907,3.729805,36.034187,36.558064,45.810001,26.794762,29.534000,26.609699,22.652231,3234.850098
2020-01-06 00:00:00+00:00,72.742638,38.631096,11.130,45.599998,48.389999,82.889999,95.143997,21.834139,0.61,283.209717,...,72.689522,3.747609,36.549992,36.792412,45.810001,27.684656,30.102667,26.134987,22.652231,3246.280029
2020-01-07 00:00:00+00:00,72.400528,38.019440,11.260,45.540001,48.250000,81.954948,95.343002,22.061375,0.60,286.285400,...,72.175163,3.676396,35.744606,36.603127,45.139999,27.839779,31.270666,25.617125,22.652231,3237.179932
2020-01-08 00:00:00+00:00,73.565193,38.631096,11.520,46.770000,47.830002,82.094215,94.598503,22.108154,0.57,289.255646,...,72.757439,3.667494,36.142769,35.521530,45.160000,27.684656,32.809334,26.109093,22.652231,3253.050049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-18 00:00:00+00:00,220.690002,32.669998,11.250,68.889999,148.289993,175.759995,186.429993,42.169998,1.01,787.840027,...,75.360001,5.030000,2.550000,41.419998,51.650002,36.970001,227.199997,29.379999,45.027000,5618.259766
2024-09-19 00:00:00+00:00,228.869995,33.650002,11.300,69.550003,156.740005,179.740005,189.869995,43.049999,1.08,828.159973,...,74.860001,5.120000,2.530000,42.750000,50.810001,37.720001,243.919998,30.320000,44.888000,5713.640137
2024-09-20 00:00:00+00:00,228.199997,34.560001,11.300,69.279999,155.949997,178.240005,191.600006,42.689999,1.10,795.280029,...,75.529999,5.080000,2.500000,42.410000,51.439999,37.209999,238.250000,29.540001,45.040001,5702.549805


In [17]:
Tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol
df_ = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[1]
df_.Date = pd.to_datetime(df_.Date.Date)
start = start_date
df_ = df_[df_.Date.Date >= start]
Tickers = Tickers[~(Tickers.isin(df_.Added.Ticker))]
Removed_Tickers = df_.Removed.Ticker
Tickers_Full = pd.concat([Tickers, Removed_Tickers])
Tickers_Full.dropna(inplace=True)
Tickers_Full.drop_duplicates(inplace=True)
Tickers_Full = list(Tickers_Full)

In [18]:
#save stocks to csv
def save_to_csv_from_yahoo(ticker, startdate, enddate):
    start = startdate
    end = enddate

    try:
        print("Get Data for :", ticker)
        df = yf.download(ticker, start=start, end=end)['Adj Close']
        time.sleep(0)
        df.to_csv('C:/Users/dietd/Untitled Folder/Python for Finance/Financial Data/Stock Data Updated/' + ticker + '.csv')

    except Exception as ex:
        stocks_not_downloaded.append(ticker)
        print("Could not download data for: "+ticker)

#function that returns df from csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', parse_dates=True)

    except FileNotFoundError:
        print("File Does Not Exist")
        pass

    else:
        return df



def update_stock_data(ticker):
    up_folder = 'C:/Users/dietd/Untitled Folder/Python for Finance/Financial Data/Stock Data Updated/'

    stock_folder = PATH

    update_df = get_df_from_csv(up_folder, ticker)
    update_df

    original_df = get_df_from_csv(stock_folder, ticker)

    #drop unnamed column
    original_df.drop(original_df.columns[original_df.columns.str.contains('unnamed', case = False)], axis=1, inplace=True)

    #drop daily return column
    original_df = original_df.drop('Daily_Return', axis=1)

    join_df = pd.concat([original_df, update_df])

    return join_df

In [19]:
def save_dataframe_to_csv(df, folder, ticker):
    df.to_csv(folder + ticker + '.csv')


def update_stocks(stock_folder):
    for x in tickers:
        try:
            print('Working on ', x)
            new_df = update_stock_data(x)
            save_dataframe_to_csv(new_df, stock_folder, x)

        except Exception as ex:
            print(ex) #prints the error type

In [20]:
#merge multiple stock in one df by column name
def merge_df_by_column_name(col_name, sdate, edate, *tick_args):
    mult_df = pd.DataFrame()
    #print(tick_args[0])
    for x in tick_args:
        #print(x)
        df = get_df_from_csv(x)
        #df['Date']=pd.to_datetime(df['Date'])
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
    
    return mult_df
        

In [21]:
prices, symbols = [], []

i=1
for symbol in Tickers_Full:
    print("Working on stock number ", i, " of ", len(Tickers_Full), " total")
    df = yf.download(symbol, start)['Adj Close']
    if not df.empty:
        prices.append(df)
        symbols.append(symbol)
    i+=1

Working on stock number  1  of  513  total


[*********************100%***********************]  1 of 1 completed


Working on stock number  2  of  513  total


[*********************100%***********************]  1 of 1 completed


Working on stock number  3  of  513  total


...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ARNC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Working on stock number  512  of  513  total


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['XEC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Working on stock number  513  of  513  total


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['WCG']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [22]:
all_prices = pd.concat(prices, axis=1)
all_prices.columns =symbols
all_prices = all_prices.dropna(axis=1)
all_prices

,MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,...,FTI,AIV,HRB,COTY,KSS,HOG,JWN,HP,CPRI,M
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,123.614922,43.839252,80.065521,72.841949,196.058365,334.429993,49.099998,17.277994,47.437042,83.347343,...,15.373728,4.859455,19.371017,10.930307,37.743382,34.255539,35.477711,35.662270,38.279999,14.130654
2020-01-03,122.550529,43.453808,79.089462,72.150536,195.731842,331.809998,48.599998,17.079590,47.108055,82.009140,...,15.553916,4.936314,19.354515,10.940200,37.804859,33.507446,35.274273,36.119160,37.380001,14.139209
2020-01-06,122.667213,43.729134,79.503830,72.719955,194.453720,333.709991,48.389999,17.277994,46.974682,82.251572,...,15.662031,4.967058,19.412268,10.732474,38.473488,33.756805,36.247231,37.143269,37.070000,14.626768
2020-01-07,122.172760,43.435455,79.061829,72.305107,190.255447,333.390015,48.250000,17.338375,46.530102,82.503677,...,15.287236,4.902688,19.404020,10.603883,38.558033,33.267311,37.140587,36.662735,37.580002,15.114326
2020-01-08,124.047607,43.371220,79.384117,72.817551,190.628677,337.869995,47.830002,17.303869,46.672363,83.318268,...,14.768293,4.924785,19.288517,10.406049,37.950878,33.738338,37.273266,35.906471,38.009998,15.482135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-18,133.360001,82.889999,114.879997,192.940002,336.179993,508.130005,148.289993,19.260000,108.959999,139.009995,...,25.360001,9.260000,62.770000,9.260000,18.969999,38.360001,22.559999,31.340000,39.070000,15.120000
2024-09-19,133.839996,84.290001,114.150002,193.619995,335.239990,526.440002,156.740005,18.900000,107.870003,141.949997,...,26.350000,9.290000,63.680000,9.390000,18.620001,39.529999,22.910000,32.240002,38.869999,15.330000
2024-09-20,134.770004,82.889999,113.699997,193.470001,336.220001,522.299988,155.949997,18.900000,107.919998,140.089996,...,27.219999,9.240000,64.459999,9.190000,18.680000,38.580002,23.020000,32.619999,38.290001,15.300000


In [23]:
port_df = test_df
mu = expected_returns.mean_historical_return(port_df)
S = risk_models.sample_cov(port_df)

In [24]:
ef = EfficientFrontier(mu, S, weight_bounds=(0,0.1))

weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()

print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.1), ('AC', 0.0), ('AGS', 0.0), ('AIR', 0.0), ('AMD', 0.04146), ('AMG', 0.05005), ('AMZN', 0.0), ('APAM', 0.0), ('ASM', 0.0), ('ASML', 0.05947), ('AVTX', 0.0), ('BAR', 0.1), ('BB', 0.0), ('BN', 0.0), ('CRBN', 0.0), ('ENX', 0.03164), ('GLPG', 0.0), ('GNFT', 0.0), ('GOOG', 0.1), ('HAL', 0.0), ('MC', 0.1), ('META', 0.09289), ('MMM', 0.0), ('NFLX', 0.0245), ('ORA', 0.0), ('SAN', 0.0), ('SAVE', 0.0), ('SLB', 0.0), ('SMAR', 0.0), ('SU', 0.0), ('TSLA', 0.1), ('UCB', 0.0), ('UMI', 0.1), ('^GSPC', 0.1)])
Expected annual return: 23.5%
Annual volatility: 24.8%
Sharpe Ratio: 0.87


(0.23541115897936862, 0.2482542543946171, 0.8677037962739518)

In [25]:
pip install pulp

In [32]:
pip install cvxpy

Note: you may need to restart the kernel to use updated packages.


In [34]:
#Get discrete allocations
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from cvxpy import GLPK
print(cleaned_weights)
portfolio_val = 52000

latest_prices = get_latest_prices(port_df)
weights = cleaned_weights

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_val)

allocation, leftover = da.greedy_portfolio()

print("Discrete Allocation: ", allocation)
print("Funds Remaining: $", leftover)

OrderedDict([('AAPL', 0.1), ('AC', 0.0), ('AGS', 0.0), ('AIR', 0.0), ('AMD', 0.04146), ('AMG', 0.05005), ('AMZN', 0.0), ('APAM', 0.0), ('ASM', 0.0), ('ASML', 0.05947), ('AVTX', 0.0), ('BAR', 0.1), ('BB', 0.0), ('BN', 0.0), ('CRBN', 0.0), ('ENX', 0.03164), ('GLPG', 0.0), ('GNFT', 0.0), ('GOOG', 0.1), ('HAL', 0.0), ('MC', 0.1), ('META', 0.09289), ('MMM', 0.0), ('NFLX', 0.0245), ('ORA', 0.0), ('SAN', 0.0), ('SAVE', 0.0), ('SLB', 0.0), ('SMAR', 0.0), ('SU', 0.0), ('TSLA', 0.1), ('UCB', 0.0), ('UMI', 0.1), ('^GSPC', 0.1)])
Discrete Allocation:  {'AAPL': 23, 'BAR': 197, 'GOOG': 31, 'MC': 73, 'TSLA': 20, 'UMI': 113, '^GSPC': 1, 'META': 8, 'ASML': 4, 'AMG': 14, 'AMD': 13, 'ENX': 167, 'NFLX': 2}
Funds Remaining: $ 11.797102928161621


In [35]:
def get_company_name_from_ticker(ticker):
    stock = yf.Ticker(ticker)
    company_info = stock.info
    company_name = company_info.get('longName', 'Company not found')
    return company_name

In [36]:
get_company_name_from_ticker("AMZN")

'Amazon.com, Inc.'

In [37]:
company_names = []
discrete_allocations = []
port_tickers = []

for ticker in allocation:
    port_tickers.append(ticker)
    company_names.append(get_company_name_from_ticker(ticker))
    discrete_allocations.append(allocation.get(ticker))

print(company_names, discrete_allocations)

['Apple Inc.', 'GraniteShares Gold Trust', 'Alphabet Inc.', 'Moelis & Company', 'Tesla, Inc.', 'USCF Midstream Energy Income Fund', 'S&P 500', 'Meta Platforms, Inc.', 'ASML Holding N.V.', 'Affiliated Managers Group, Inc.', 'Advanced Micro Devices, Inc.', 'Eaton Vance New York Municipal Bond Fund', 'Netflix, Inc.'] [23, 197, 31, 73, 20, 113, 1, 8, 4, 14, 13, 167, 2]


In [38]:
portfolio_df = pd.DataFrame(columns=['Company Name', "Ticker", "Discrete Allocation "+str(portfolio_val) ])
portfolio_df

,Company Name,Ticker,Discrete Allocation 52000


In [39]:
portfolio_df['Company Name']=company_names
portfolio_df['Ticker']= port_tickers
portfolio_df["Discrete Allocation "+str(portfolio_val)] = discrete_allocations

In [40]:
portfolio_df

,Company Name,Ticker,Discrete Allocation 52000
0,Apple Inc.,AAPL,23
1,GraniteShares Gold Trust,BAR,197
2,Alphabet Inc.,GOOG,31
3,Moelis & Company,MC,73
4,"Tesla, Inc.",TSLA,20
5,USCF Midstream Energy Income Fund,UMI,113
6,S&P 500,^GSPC,1
7,"Meta Platforms, Inc.",META,8
8,ASML Holding N.V.,ASML,4
9,"Affiliated Managers Group, Inc.",AMG,14
